
# Pracownia nr 2

In [81]:
using Polynomials

## Korzystanie ze wzoru na przybliżoną wartość pochodnej

### Znajdowanie najmniejszego przedziału między węzłami, do którego należy x

In [82]:
function search_bounds(x_p, x_k, x, nodes)
    if x_p + 1 == x_k
        return nodes[x_p], nodes[x_k]
    else
        if x > nodes[div((x_p + x_k), 2)]
            search_bounds(div((x_p + x_k), 2), x_k, x, nodes)
        else
            search_bounds(x_p, div((x_p + x_k), 2), x, nodes)
        end
    end
end;      

In [83]:
search_bounds(1, 3, 1.3, [1, 2, 3])

(1, 2)

### Wzór na przybliżoną wartość pochodnej dla x z przedziału (x_p, x_k)

In [84]:
function approx_derivate(f, nodes, x)
    interval = search_bounds(1, length(nodes), x, nodes)
    x_p = BigFloat(interval[1])
    x_k = BigFloat(interval[2])
    
    return Float64((f(x_k) - f(x_p)) / (x_k - x_p))
end;

In [85]:
approx_derivate(x -> x^2, [1, 2, 3], 2.5)

5.0

## Pochodna wielomianu interpolacyjnego w postaci Newtona

### wyliczanie kolejnych ilorazów różnicowych

In [86]:
function coefs(nodes, f) 
    all_coefs = [];
    
    quots = map(f, nodes)
    push!(all_coefs, quots[1])
    
    len = length(nodes)
    for i = 1:(len-1)
        for j = 1:(len-i)
           quots[j] = (quots[j+1] - quots[j] ) / (nodes[j+i] - nodes[j])
        end
        push!(all_coefs, quots[1])
    end
    
    return all_coefs
end;

In [87]:
coefs([1, 2, 3], x -> x^2)

3-element Array{Any,1}:
 1
 3
 1

### wyliczenie wielomianu interpolacyjnego Newtona na podstawie węzłów

In [88]:
function newton_interpolation(nodes, f)
    nodes_number = length(nodes)
    all_coefs = coefs(nodes, f)
    
    newton = Poly([all_coefs[1]])
    actual_poly = Poly([1])
    for i in 2:nodes_number
        actual_poly *= Poly([-nodes[i-1], 1])
        newton += actual_poly * all_coefs[i] 
    end
    
    return newton
end;

In [89]:
w = newton_interpolation([1, 2, 3], x -> sin(x))

Poly(-0.06235931799348837 + 1.3218322331931847*x - 0.41800193039179984*x^2)

In [90]:
polyval(w, pi)

-0.0352145767743442

### wyliczenie pochodnej w punkcie

In [91]:
function newton_derivate(f, nodes, point)
    return Float64(polyval(polyder(newton_interpolation(nodes, f)), BigFloat(point)))
end;

In [92]:
newton_derivate(x -> sin(x), [1, 2, 3], 0.0121)

1.3117165864777032

In [93]:
newton_derivate(x->log(x), [1, 2, 3], 1.21)

0.7765749815709618

## Testowanie metod dla róznego doboru węzłów

#### Węzły równoodległe na przedziale (0, 10), liczba węzłów: 6, odległość między węzłami: 2

In [94]:
nodes_2 = linspace(BigFloat(0), BigFloat(10), 6);

#### Węzły równoodległe na przedziale (0, 10), liczba węzłów: 11, odległość między węzłami: 1

In [95]:
nodes_1 = linspace(BigFloat(0), BigFloat(10), 11);

#### Węzły równoodległe na przedziale (0, 10), liczba węzłów: 101, odległość między węzłami: 0.1

In [96]:
nodes_01 = linspace(BigFloat(0), BigFloat(10), 101);

#### Węzły równoodległe na przedziale (0.01, 10), liczba węzłów: 1000, odległość między węzłami: 0.01

In [97]:
nodes_001 = linspace(BigFloat(0.01), BigFloat(10), 1000);

### testowanie dla różych węzłów

In [98]:
function test_nodes(f, nodes1, nodes2, nodes3, x)
    return [approx_derivate(f, nodes1, x) approx_derivate(f, nodes2, x) approx_derivate(f, nodes3, x) newton_derivate(f, nodes1, x) newton_derivate(f, nodes2, x) newton_derivate(f, nodes3, x)]
end;

In [99]:
function relative_errors(p, f, nodes1, nodes2, nodes3, x)
    return map(y -> abs((y - p(x)) / p(x)), [approx_derivate(f, nodes1, x) approx_derivate(f, nodes2, x) approx_derivate(f, nodes3, x) newton_derivate(f, nodes1, x) newton_derivate(f, nodes2, x) newton_derivate(f, nodes3, x)])    
end;

#### dla funkcji f(x) = sin(x), w punktach 𝛑, 6.12, 1.1

##### 𝛑

In [100]:
test_nodes(x -> sin(x), nodes_2, nodes_1, nodes_01, pi)

1×6 Array{Float64,2}:
 -0.83305  -0.897923  -0.999548  -0.866058  -1.00005  -1.0

In [101]:
cos(pi)      # wartość dokładna

-1.0

In [102]:
relative_errors(x -> cos(x), x -> sin(x), nodes_2, nodes_1, nodes_01, pi)

1×6 Array{Float64,2}:
 0.16695  0.102077  0.000451941  0.133942  5.35236e-5  0.0

##### 6.12

In [103]:
test_nodes(x -> sin(x), nodes_2, nodes_1, nodes_01, 6.12)

1×6 Array{Float64,2}:
 0.634387  0.936402  0.990731  0.732653  0.986834  0.986715

In [104]:
cos(6.12)      # wartość dokładna

0.9867147985168921

In [105]:
relative_errors(x -> cos(x), x-> sin(x), nodes_2, nodes_1, nodes_01, 6.12)

1×6 Array{Float64,2}:
 0.357072  0.0509901  0.00407029  0.257483  0.000121247  0.0

#### 9.9

In [106]:
test_nodes(x -> sin(x), nodes_2, nodes_1, nodes_01, 9.9)

1×6 Array{Float64,2}:
 -0.76669  -0.95614  -0.864852  -1.83211  -0.869791  -0.889191

In [107]:
cos(1.1)

0.4535961214255773

In [108]:
relative_errors(x -> cos(x), x -> sin(x), nodes_2, nodes_1, nodes_01, 9.9)

1×6 Array{Float64,2}:
 0.137767  0.0752914  0.027372  1.06043  0.0218174  0.0

#### dla funkcji f(x) = exp(x), w punktach 1.2, 𝛑, 30

#### 1.2

In [109]:
test_nodes(x -> exp(x), nodes_2, nodes_1, nodes_01, 1.2)

1×6 Array{Float64,2}:
 3.19453  4.67077  3.15951  -225.744  3.84198  3.32012

In [110]:
exp(1.2)    # wartość dokładna

3.3201169227365472

In [111]:
relative_errors(x -> exp(x), x -> exp(x), nodes_2, nodes_1, nodes_01, 1.2)

1×6 Array{Float64,2}:
 0.0378266  0.40681  0.0483742  68.9927  0.157182  0.0

#### 𝛑

In [112]:
test_nodes(x -> exp(x), nodes_2, nodes_1, nodes_01, pi)

1×6 Array{Float64,2}:
 23.6045  34.5126  23.3458  74.7069  23.2431  23.1407

In [113]:
exp(pi)    # wartość dokładna

23.140692632779267

In [114]:
relative_errors(x -> exp(x), x -> exp(x), nodes_2, nodes_1, nodes_01, pi)

1×6 Array{Float64,2}:
 0.020045  0.491425  0.00886302  2.22838  0.00442462  1.53527e-16

#### 30

In [115]:
approx_derivate(x -> exp(x), linspace(29, 31, 100), 30)

1.0686656306673238e13

In [116]:
newton_derivate(x -> exp(x), linspace(29, 31, 100), 30)

-3.288454563525524e197

In [117]:
exp(30)    # wartość dokładna

1.0686474581524463e13

In [118]:
abs((1.0686656306673238e13 - exp(30)) / exp(30))^

LoadError: [91msyntax: incomplete: premature end of input[39m

#### dla funkcji f(x) = 2*(x + 1)*(x - 3)

In [119]:
f(x) = 2*(x + 1) * (x - 3);
p(x) = 4*x - 4;    # pochodna funkcji f

#### 1

In [120]:
test_nodes(x -> 2 * (x + 1) * (x - 3), nodes_2, nodes_1, nodes_01, 1)

1×6 Array{Float64,2}:
 0.0  -2.0  -0.2  0.0  0.0  4.2648e-61

In [121]:
p(1)    # wartość dokładna

0

In [123]:
# liczenie błędów względnych wymagałoby dzielenia przez 0

#### 5.123

In [124]:
test_nodes(x-> 2 * (x + 1) * (x - 3), nodes_2, nodes_1, nodes_01, 5.123)

1×6 Array{Float64,2}:
 16.0  18.0  16.6  16.492  16.492  16.492

In [125]:
p(5.123)

16.492

In [126]:
relative_errors(x -> 4 * x - 4, x -> 2 * (x + 1) * (x - 3) , nodes_2, nodes_1, nodes_01, 5.123)

1×6 Array{Float64,2}:
 0.0298326  0.0914383  0.00654863  0.0  0.0  0.0

#### 9.9

In [127]:
test_nodes(x-> 2 * (x + 1) * (x - 3), nodes_2, nodes_1, nodes_01, 9.9)

1×6 Array{Float64,2}:
 32.0  34.0  35.8  35.6  35.6  35.6

In [128]:
4 * 9.9 - 4    # wartość dokładna

35.6

In [129]:
relative_errors(x -> 4 * x - 4, x -> 2 * (x + 1) * (x - 3) , nodes_2, nodes_1, nodes_01, 9.9)

1×6 Array{Float64,2}:
 0.101124  0.0449438  0.00561798  0.0  0.0  0.0

#### dla funkcji f(x) = ln(x), w punktach 0.1, 1

#### 0.1

In [130]:
test_nodes(x -> log(x), nodes_2, nodes_1, nodes_01, 0.1)

1×6 Array{Float64,2}:
 Inf  Inf  6.93147  NaN  NaN  NaN

In [131]:
test_nodes(x -> log(x), linspace(0.1, 10, 6), linspac w e(0.5, 10, 6), nodes_001, 0.1)

1×6 Array{Float64,2}:
 1.5328  0.825587  9.53102  2.91226  1.71225  8.08341e200

In [132]:
1 / 0.1    # wartość dokładna

10.0

In [133]:
relative_errors(x -> 1/x, x -> log(x) , linspace(0.1, 10, 6), linspace(0.5, 10, 6), nodes_001, 0.1)

1×6 Array{Float64,2}:
 0.84672  0.917441  0.0468982  0.708774  0.828775  8.08341e199

In [136]:
test_nodes(x -> log(x), linspace(0.1, 10, 6), linspace(0.5, 10, 6), nodes_001, 1)

1×6 Array{Float64,2}:
 1.5328  0.825587  1.00503  1.51598  1.02121  -1.41582e292

In [ ]:
1 / 1    # wartość dokładna

In [135]:
relative_errors(x -> 1/x, x -> log(x) , linspace(0.1, 10, 6), linspace(0.5, 10, 6), nodes_001, 1)

1×6 Array{Float64,2}:
 0.532805  0.174413  0.00503359  0.515983  0.0212106  1.41582e292